# Some experiments with OpenCV

In [ ]:
import cv2
import io
import numpy as np
import urllib.request

from PIL import Image as PImage

In [ ]:
# helper functions to convert between PIL and OpenCV image types

def tocv(pil):
  return np.array(pil)

def topil(cv):
  return PImage.fromarray(cv)

## Open an Image

In [ ]:
# open image from url
response = urllib.request.urlopen("https://raw.githubusercontent.com/PGDV-5200-2025F-A/silhouettes/refs/heads/main/imgs/00_original/01360.jpg")
image_data = io.BytesIO(response.read())
img = PImage.open(image_data)

# this makes the largest edge of the image be 480
img.thumbnail((480, 480))
display(img)

## Threshold

Turn color/gray image into black and white

Doc: https://docs.opencv.org/4.x/db/d8e/tutorial_threshold.html

In [ ]:
# opencv functions take raw pixel arrays and not PIL images 🤷

img_cv = tocv(img)

ret, img_thold_cv = cv2.threshold(cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY)

display(topil(img_thold_cv))

## Erode / Dilate

Reduces / Expands white regions on the image, respectively.

By applying complementary erode/dilate operations you can get rid of gaps and concave parts of an image.

Doc: https://docs.opencv.org/4.x/db/df6/tutorial_erosion_dilatation.html

In [ ]:
# this sets up the shape and size of the erosion filter
eksize = 2
ekernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * eksize + 1, 2 * eksize + 1), (eksize, eksize))

dksize = 3
dkernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * dksize + 1, 2 * dksize + 1), (dksize, dksize))

eroded_cv = cv2.erode(img_thold_cv, ekernel)
dilated_cv = cv2.dilate(eroded_cv, dkernel)

display(topil(dilated_cv))

## Extracting Outline

The function for this is called `findContours()`.

Docs: https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html

In [ ]:
colors = [(220,0,0),(0,220,0),(0,0,220),(220,220,0),(0,220,220),(220,0,220)]

draw_cv = cv2.cvtColor(dilated_cv.copy(), cv2.COLOR_GRAY2RGB)

contours, hierarchy = cv2.findContours(image=dilated_cv, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

if contours:
  for idx,con in enumerate(contours):
    cv2.drawContours(draw_cv, [con], 0, colors[idx%len(colors)], 2)

display(PImage.fromarray(draw_cv))

## Contour objects

are just lists of x,y coordinates, but for some reason opencv adds a mysterious dimension to them.

Instead of this:

```python
[
  [x0,y0], [x1,y1], [x2,y2], ...
]
```

We get this (double array around points):
```python
[
  [[x0,y0]], [[x1,y1]], [[x2,y2]], ...
]
```

We can fix it by using the `squeeze()` function which gets rid of superfluous dimensions in arrays:

In [ ]:
# if we want the first contour, this will turn it into a plain list of (x,y) coordinates
contours[0].squeeze().tolist()

### Contour Area

The `contourArea()` function gives the area of a contour. Useful when trying to find the largest contour in an image.

Docs: https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html

In [ ]:
cv2.contourArea(contours[0])

## Filtering

I used this function to filter the silhouette contours. Contours with points touching the edges of the image, or contours larger than 80% of the image area, or contours smaller than 5% of the image area, are not valid.

In [ ]:
def contour_is_valid(c, h, w, m=1):
  for p in c:
    x, y = p[0]
    if x < m or x > w - m - 1 or y < m or y > h - m - 1:
      return False
  return (cv2.contourArea(c) < 0.80 * h * w) and (cv2.contourArea(c) > 0.05 * h * w)

contour_is_valid(contours[0], img.size[1], img.size[0])

In [ ]:
colors = [(220,0,0),(0,220,0),(0,0,220),(220,220,0),(0,220,220),(220,0,220)]

draw_cv = img_cv.copy()

contours, hierarchy = cv2.findContours(image=dilated_cv, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

valid_contours = [con for con in contours if contour_is_valid(con, img.size[1], img.size[0])]

for idx,con in enumerate(valid_contours):
  cv2.drawContours(draw_cv, [con], 0, colors[idx%len(colors)], 2)

display(PImage.fromarray(draw_cv))

## Feature Points

Not sure if these will help, necessarily, but was something I thought about in terms of extracting features from images.

Docs: https://docs.opencv.org/4.x/db/d27/tutorial_py_table_of_contents_feature2d.html

### SIFT

Docs: https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html

In [ ]:
img_cv = tocv(img)

draw_cv = img_cv.copy()

gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)

sift = cv2.SIFT_create(32)
kp = sift.detect(gray_cv, None)
 
cv2.drawKeypoints(draw_cv, kp, draw_cv, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

topil(draw_cv)

### FAST

Docs: https://docs.opencv.org/4.x/df/d0c/tutorial_py_fast.html

In [ ]:
img_cv = tocv(img)

draw_cv = img_cv.copy()

gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)

fast = cv2.FastFeatureDetector_create(threshold=80)
kp = fast.detect(gray_cv, None)

cv2.drawKeypoints(draw_cv, kp, draw_cv, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

topil(draw_cv)